In [ ]:
import pandas as pd

In [ ]:
flight_df_raw = pd.read_csv(
    r"C:\Users\macie\OneDrive\Pulpit\Studia PJATK\flight.csv",
    low_memory=False)


In [ ]:
flight_df_raw

In [ ]:
flight_df_raw.info()
flight_df_raw.columns

In [ ]:
flight_df_raw = flight_df_raw.drop('Unnamed: 32', axis=1)
flight_df_raw

In [ ]:
flight_df_raw['CANCELLATION_CODE'] == 'NaN'


In [ ]:
flight_df_raw = flight_df_raw.drop('CANCELLATION_CODE', axis=1)

In [ ]:
flight_df_raw

testy hipotetyczne w jakie dni tygodnia spozniaja sie te loty 
korelacje między pogodą 

Usuwamy loty anulowanae 

In [ ]:
cancelled_flights = flight_df_raw['CANCELLED'] == 1

first_cl_flight_df_raw = flight_df_raw[~(cancelled_flights)]
first_cl_flight_df_raw

Zmieniamy nazwe kolumny DEP_DELAY_NEW na DEP_DELAY

In [ ]:
first_cl_flight_df_raw.rename(columns = {'DEP_DELAY_NEW': 'DEP_DELAY'}, inplace = True)
flight_df = first_cl_flight_df_raw

In [ ]:
flight_df['DEP_DELAY']

## Analiza kolumny dep_delay part 1
Chcemy wyznaczyć statystyki opisowe dla DEP_DELAY i zapisać do nowej zmiennej. 
Oto wiersze, które chcemy uzyskać:
-  średnia,
 - mediana,
 - odchylenie standardowe,
 - min, max
 - percentyle `[0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99]`


In [ ]:
dep_delay_statistics_df = round(flight_df['DEP_DELAY'].describe(percentiles = [0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99]), 2)

In [ ]:
dep_delay_statistics_df

## Analiza kolumny DEP_DELAY part 2
Chcemy podać dokładnej analizie tę kolumne:
1. Wyznaczyć wykres dla całej kolumny (tzn. tak jak jest)
2. Wyznaczyć wykres z pominięciem tych wierszy, dla których DEP_DELAY = 0
3. Ograniczyć wykres do percentyla 95% oraz pomijając DEP_DELAY = 0

In [ ]:
flight_df['DEP_DELAY']

In [ ]:
import plotly.express as px
import matplotlib.pyplot as plt

In [ ]:
fig1 = px.histogram(flight_df, x='DEP_DELAY', nbins=int(flight_df['DEP_DELAY'].max() / 10))

fig1.show()

Wykres dla DEP_DELAY > 0

In [ ]:
fig2 = px.histogram(flight_df.loc[flight_df['DEP_DELAY'] > 0], x='DEP_DELAY', nbins=int(flight_df['DEP_DELAY'].max() / 10))
fig2.show()

 wykres dla dep_delay używając warunków dep_delay > 0 oraz dep_delay < percentile 95%

In [ ]:
import plotly.graph_objects as go
percentile_95 = dep_delay_statistics_df.loc['95%']

filtered_delays = flight_df[(flight_df['DEP_DELAY'] > 0) & (flight_df['DEP_DELAY'] < percentile_95)]

fig3 = go.Figure(data=[go.Histogram(x=filtered_delays['DEP_DELAY'], 
                                   xbins=dict(start=0, end=filtered_delays['DEP_DELAY'].max(), size=10))])


fig3.show()

Utworzymy w ramce flight_df nową kolumnę - IS_DELAYED jako te opóźnienia, które wynosiły więcej niż (>) 15 minut.
Chcemy wyznaczyć jaki procent wszystkich lotów był opóźniony.

In [ ]:
import numpy as np
flight_df['IS_DELAYED'] = np.where(flight_df['DEP_DELAY'] > 15, 1, 0)

In [ ]:
delayed_ratio = flight_df['IS_DELAYED'].mean()
delayed_ratio = round(delayed_ratio, 2)
delayed_ratio

## Opóźnienia  vs Miesiąc kalendarzowy
### Zbadamy jak zmienia się odsetek opóźnień w zależności od miesiąca kalendarzowego. 

In [ ]:
flight_delays_by_month_df = flight_df.groupby('MONTH')['IS_DELAYED'].mean()
flight_delays_by_month_df

In [ ]:
fig4 = px.bar(flight_delays_by_month_df, title = 'delays vs month')
fig4.show()

## Opóźnienia vs dzień tygodnia part 1
Chcemy zbadać, jak zmienia się odsetek opóźnień w zależności od dnia tygodnia.

In [ ]:
flight_delays_by_weekday_df = flight_df.groupby('DAY_OF_WEEK')['IS_DELAYED'].mean()
flight_delays_by_weekday_df

In [ ]:
fig5 = px.bar(flight_delays_by_weekday_df, title = 'delays vs day_of_week')
fig5.show()

## Opóźnienia vs dzień tygodnia part 2
Dokonamy agregacji kolumny DAY_OF_WEEK do nowej kolumny IS_WEEKEND w flight_df. Wartości, które reprezentują weekend to 6 i 7.

In [ ]:
flight_df['IS_WEEKEND'] = np.where(flight_df['DAY_OF_WEEK'] <= 5, 'not weekend', 'is weekend')

In [ ]:
flight_delays_by_weekend_df = round(flight_df.loc[flight_df['IS_DELAYED'] == 1].groupby(by = 'IS_WEEKEND')['IS_DELAYED'].count()/flight_df.groupby(by = 'IS_WEEKEND')['IS_DELAYED'].count(), 2)
flight_delays_by_weekend_df

In [ ]:
fig6 = px.bar(flight_delays_by_weekend_df, title = 'delays/weekend or not weekend')
fig6.show()

## Opoźnienia vs odległość lotu 
### 1. W tej części chcemy wyznaczyć statystyki opisowe dla kolumny 'distance' wzbogacone o percentyle. 

In [ ]:
flight_distance_analysis_df = round(flight_df['DISTANCE'].describe(percentiles = [0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99]), 2)
flight_distance_analysis_df


In [ ]:
df10k = flight_df[['DISTANCE', 'DEP_DELAY']].sample(n = 10000)
fig = px.scatter(df10k, x = 'DEP_DELAY', y = 'DISTANCE')
fig.show()

In [ ]:
flight_df.drop(flight_df.loc[flight_df['DISTANCE'] > flight_distance_analysis_df[9]].index, axis = 0, inplace = True)

In [ ]:
flight_df['DISTANCE_AGG'] = pd.cut(flight_df['DISTANCE'], bins = [*range(0, int(flight_df['DISTANCE'].max()), 100)])


In [ ]:
flight_delays_by_distance_agg_df = flight_df.loc[flight_df['IS_DELAYED'] == 1].groupby(by = 'DISTANCE_AGG')['IS_DELAYED'].count()/flight_df.groupby(by = 'DISTANCE_AGG')['IS_DELAYED'].count()

In [ ]:
flight_delays_by_distance_agg_df = pd.DataFrame(flight_delays_by_distance_agg_df).sort_values(by = 'DISTANCE_AGG')
flight_delays_by_distance_agg_df = flight_delays_by_distance_agg_df.fillna(0)
flight_delays_by_distance_agg_df['INDEX_COLUMN'] = flight_delays_by_distance_agg_df.index

In [ ]:
plt.figure(figsize=(15,10))
plt.xticks(rotation=90)
plt.bar(x = flight_delays_by_distance_agg_df['INDEX_COLUMN'].astype(str), height = flight_delays_by_distance_agg_df['IS_DELAYED'])
plt.show()



## Opóźnieni vs grupa odległości 

In [ ]:
distance_gr = flight_df['DISTANCE_GROUP'].unique()
print(distance_gr)

In [ ]:
min_dg = []  
max_dg = []
for p in distance_gr:
    min_dg_temp = flight_df['DISTANCE'].loc[flight_df['DISTANCE_GROUP']==p].min()
    max_dg_temp = flight_df['DISTANCE'].loc[flight_df['DISTANCE_GROUP']==p].max()
    min_dg.append(min_dg_temp)
    max_dg.append(max_dg_temp)

In [ ]:
distance_gr_dict ={
    'group': distance_gr,
    'min': min_dg,
    'max': max_dg
}

In [ ]:
flight_distance_by_distance_group_df = pd.DataFrame.from_dict(distance_gr_dict)
flight_distance_by_distance_group_df = flight_distance_by_distance_group_df.sort_values('group')
flight_distance_by_distance_group_df = flight_distance_by_distance_group_df.reset_index()
flight_distance_by_distance_group_df.drop('index', axis = 1, inplace = True)
flight_distance_by_distance_group_df

In [ ]:
flight_delays_by_distance_group_df = flight_df.loc[flight_df['IS_DELAYED'] == 1].groupby(by = 'DISTANCE_GROUP')['IS_DELAYED'].count()/flight_df.groupby(by = 'DISTANCE_GROUP')['IS_DELAYED'].count()
flight_delays_by_distance_group_df

In [ ]:
fig = px.bar(flight_delays_by_distance_group_df)
fig.show()

### Czy większy dystans oznacza większe prawdopodobieństwo opóźnienia lotu? 
Czy możemy porównać wykresy?

Różnica wartości zaimplementowanych koszyków nie pozwala na jednoznaczne porównanie poszczególnych wykresów. Możliwe opóźnienie nie jest efektem długości lotu. 

sprawdzic np srednia w danym + wykorzystac metodyke